In [1]:
import os
import time
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.mass_downloader import (
    RectangularDomain, Restrictions, MassDownloader
)
from http.client import IncompleteRead
from obspy.clients.fdsn.header import FDSNException

In [ ]:
import os
import time
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.mass_downloader import (
    RectangularDomain, Restrictions, MassDownloader
)
from http.client import IncompleteRead
from obspy.clients.fdsn.header import FDSNException

def get_java_earthquake_catalog(start_date, end_date, region_params, max_retries=5, chunk_size=100):
    client = Client("IRIS", timeout=120)
    catalog = obspy.Catalog()
    offset = 0
    while True:
        for attempt in range(max_retries):
            try:
                chunk = client.get_events(
                    starttime=start_date,
                    endtime=end_date,
                    minlatitude=region_params['min_lat'],
                    maxlatitude=region_params['max_lat'],
                    minlongitude=region_params['min_lon'],
                    maxlongitude=region_params['max_lon'],
                    minmagnitude=5,
                    orderby="time",
                    limit=chunk_size,
                    offset=offset
                )
                catalog += chunk
                offset += chunk_size
                if len(chunk) < chunk_size:
                    return catalog
                break
            except (IncompleteRead, FDSNException):
                if attempt < max_retries - 1:
                    time.sleep(2 ** attempt)
                else:
                    return catalog
        else:
            break
    return catalog

def download_event_data(event, base_dir, retries=3):
    event_id = str(event.resource_id).split('/')[-1]
    event_dir = os.path.join(base_dir, f"event_{event_id}")
    origin = event.preferred_origin() or event.origins[0]
    domain = RectangularDomain(
        minlatitude=origin.latitude - 2,
        maxlatitude=origin.latitude + 2,
        minlongitude=origin.longitude - 2,
        maxlongitude=origin.longitude + 2
    )
    restrictions = Restrictions(
        starttime=origin.time - 60,
        endtime=origin.time + 300,
        reject_channels_with_gaps=True,
        minimum_length=0.85,
        channel_priorities=["HH[ZNE]", "BH[ZNE]", "HN[ZNE]"],
        location_priorities=["", "00", "10"]
    )
    for attempt in range(retries):
        try:
            mdl = MassDownloader(providers=["IRIS"])
            mdl.download(
                domain,
                restrictions,
                mseed_storage=os.path.join(event_dir, "waveforms"),
                stationxml_storage=os.path.join(event_dir, "stations")
            )
            print(f"Downloaded data for event {event_id}")
            return True
        except (IncompleteRead, FDSNException):
            if os.path.exists(event_dir):
                for f in os.listdir(event_dir):
                    os.remove(os.path.join(event_dir, f))
                os.rmdir(event_dir)
            time.sleep(2 ** attempt)
    return False

def main():
    config = {
        'region': {
            'min_lat': -10.0,
            'max_lat': -5.0,
            'min_lon': 100.0,
            'max_lon': 120.0
        },
        'time_range': {
            'start': UTCDateTime("1800-06-17T06:11:00"),
            'end': UTCDateTime("2025-04-11T04:14:00")
        },
        'output_dir': "New_java_earthquake_data",
        'max_events': 10000
    }
    os.makedirs(config['output_dir'], exist_ok=True)
    catalog = get_java_earthquake_catalog(
        config['time_range']['start'],
        config['time_range']['end'],
        config['region']
    )
    catalog.events = sorted(catalog, 
        key=lambda x: x.origins[0].time, 
        reverse=True
    )[:config['max_events']]
    catalog_file = os.path.join(config['output_dir'], "event_catalog.xml")
    catalog.write(catalog_file, format="QUAKEML")
    for i, event in enumerate(catalog):
        if download_event_data(event, config['output_dir']):
            pass  # The print is inside download_event_data

if __name__ == "__main__":
    main()


[2025-05-01 12:05:20,887] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for IRIS.
[2025-05-01 12:05:20,891] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): IRIS.
[2025-05-01 12:05:20,892] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2025-05-01 12:05:20,892] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Requesting reliable availability.
[2025-05-01 12:05:21,338] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available for request.
[2025-05-01 12:05:21,339] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available.
[2025-05-01 12:05:21,340] - obspy.clients.fdsn.mass_downloader - INFO: ============================== Final report
[2025-05-01 12:05:21,340] - obspy.clients.fdsn.mass_downloader - INFO: 0 MiniSEED files [0.0 MB] already existed.
[2025-05-01 12:05:21,340] - obspy.clients.fdsn.mass_downloader - INFO: 0 StationXML files

Downloaded data for event query?eventid=11938895


[2025-05-01 12:05:21,798] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available for request.
[2025-05-01 12:05:21,798] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available for request.
[2025-05-01 12:05:21,799] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available.
[2025-05-01 12:05:21,799] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available.
[2025-05-01 12:05:21,800] - obspy.clients.fdsn.mass_downloader - INFO: ============================== Final report
[2025-05-01 12:05:21,800] - obspy.clients.fdsn.mass_downloader - INFO: ============================== Final report
[2025-05-01 12:05:21,801] - obspy.clients.fdsn.mass_downloader - INFO: 0 MiniSEED files [0.0 MB] already existed.
[2025-05-01 12:05:21,801] - obspy.clients.fdsn.mass_downloader - INFO: 0 MiniSEED files [0.0 MB] already existed.
[2025-05-01 12:05:21,802] - obspy.clients.fdsn.mass_downloader - INFO: 0 StationXML files [0.0

Downloaded data for event query?eventid=11938890


[2025-05-01 12:05:22,268] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available for request.
[2025-05-01 12:05:22,268] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available for request.
[2025-05-01 12:05:22,268] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available for request.
[2025-05-01 12:05:22,270] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available.
[2025-05-01 12:05:22,270] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available.
[2025-05-01 12:05:22,270] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available.
[2025-05-01 12:05:22,271] - obspy.clients.fdsn.mass_downloader - INFO: ============================== Final report
[2025-05-01 12:05:22,271] - obspy.clients.fdsn.mass_downloader - INFO: ============================== Final report
[2025-05-01 12:05:22,271] - obspy.clients.fdsn.mass_downloader - INFO: ===========================

...

[2025-05-01 12:05:22,741] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available for request.
[2025-05-01 12:05:22,741] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available for request.
[2025-05-01 12:05:22,741] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available for request.
[2025-05-01 12:05:22,741] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available for request.
[2025-05-01 12:05:22,743] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available.
[2025-05-01 12:05:22,743] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available.
[2025-05-01 12:05:22,743] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available.
[2025-05-01 12:05:22,743] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - No data available.
[2025-05-01 12:05:22,746] - obspy.clients.fdsn.mass_downloader - INFO: ============================== Fi

Downloaded data for event query?eventid=11934135


[2025-05-01 12:05:23,245] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully requested availability (0.44 seconds)
[2025-05-01 12:05:23,245] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully requested availability (0.44 seconds)
[2025-05-01 12:05:23,245] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully requested availability (0.44 seconds)
[2025-05-01 12:05:23,245] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully requested availability (0.44 seconds)
[2025-05-01 12:05:23,245] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully requested availability (0.44 seconds)
[2025-05-01 12:05:23,249] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Found 0 stations (0 channels).
[2025-05-01 12:05:23,249] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Found 0 stations (0 channels).
[2025-05-01 12:05:23,249] - obspy.clients.fdsn.mass_downloader - INFO: Cli

Downloaded data for event query?eventid=11933296


[2025-05-01 12:05:24,148] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully requested availability (0.86 seconds)
[2025-05-01 12:05:24,148] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully requested availability (0.86 seconds)
[2025-05-01 12:05:24,148] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully requested availability (0.86 seconds)
[2025-05-01 12:05:24,148] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully requested availability (0.86 seconds)
[2025-05-01 12:05:24,148] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully requested availability (0.86 seconds)
[2025-05-01 12:05:24,148] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully requested availability (0.86 seconds)
[2025-05-01 12:05:24,153] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Found 1 stations (3 channels).
[2025-05-01 12:05:24,153] - obspy.clients.fdsn.mass_do

In [ ]:
# import os
# import time
# import obspy
# import logging
# from obspy import UTCDateTime
# from obspy.clients.fdsn import Client
# from obspy.clients.fdsn.mass_downloader import (
#     RectangularDomain, Restrictions, MassDownloader
# )
# from http.client import IncompleteRead
# from obspy.clients.fdsn.header import FDSNException

# # Configure logging
# logging.basicConfig(
#     level=logging.INFO,
#     format='%(asctime)s - %(levelname)s - %(message)s',
#     handlers=[logging.FileHandler('earthquake_download.log'), logging.StreamHandler()]
# )

# def get_java_earthquake_catalog(start_date, end_date, region_params, max_retries=5, chunk_size=100):
#     """
#     Retrieve earthquake catalog with pagination and error handling
#     """
#     client = Client("IRIS", timeout=120)
#     catalog = obspy.Catalog()
#     offset = 0
    
#     while True:
#         for attempt in range(max_retries):
#             try:
#                 chunk = client.get_events(
#                     starttime=start_date,
#                     endtime=end_date,
#                     minlatitude=region_params['min_lat'],
#                     maxlatitude=region_params['max_lat'],
#                     minlongitude=region_params['min_lon'],
#                     maxlongitude=region_params['max_lon'],
#                     orderby="time",
#                     limit=chunk_size,
#                     offset=offset
#                 )
#                 catalog += chunk
#                 offset += chunk_size
#                 logging.info(f"Retrieved {len(chunk)} events (total: {len(catalog)})")
#                 if len(chunk) < chunk_size:
#                     return catalog
#                 break
#             except (IncompleteRead, FDSNException) as e:
#                 logging.warning(f"Attempt {attempt+1} failed: {str(e)}")
#                 if attempt < max_retries - 1:
#                     delay = 2 ** attempt
#                     logging.info(f"Retrying in {delay} seconds...")
#                     time.sleep(delay)
#                 else:
#                     logging.error("Max retries reached for catalog download")
#                     return catalog
#         else:
#             break
#     return catalog

# def download_event_data(event, base_dir, retries=3):
#     """
#     Download waveform data for a single event with retries
#     """
#     event_id = str(event.resource_id).split('/')[-1]
#     event_dir = os.path.join(base_dir, f"event_{event_id}")
#     origin = event.preferred_origin() or event.origins[0]
    
#     domain = RectangularDomain(
#         minlatitude=origin.latitude - 2,
#         maxlatitude=origin.latitude + 2,
#         minlongitude=origin.longitude - 2,
#         maxlongitude=origin.longitude + 2
#     )
    
#     restrictions = Restrictions(
#         starttime=origin.time - 60,
#         endtime=origin.time + 300,
#         reject_channels_with_gaps=True,
#         minimum_length=0.85,
#         channel_priorities=["HH[ZNE]", "BH[ZNE]", "HN[ZNE]"],
#         location_priorities=["", "00", "10"]
#     )
    
#     for attempt in range(retries):
#         try:
#             mdl = MassDownloader(providers=["IRIS"])
#             mdl.download(
#                 domain,
#                 restrictions,
#                 mseed_storage=os.path.join(event_dir, "waveforms"),
#                 stationxml_storage=os.path.join(event_dir, "stations")
#             )
#             logging.info(f"Successfully downloaded {event_id}")
#             return True
#         except (IncompleteRead, FDSNException) as e:
#             logging.warning(f"Attempt {attempt+1} failed for {event_id}: {str(e)}")
#             if os.path.exists(event_dir):
#                 for f in os.listdir(event_dir):
#                     os.remove(os.path.join(event_dir, f))
#                 os.rmdir(event_dir)
#             time.sleep(2 ** attempt)
#     logging.error(f"Failed to download {event_id} after {retries} attempts")
#     return False

# def main():
#     # Configuration parameters
#     config = {
#         'region': {
#             'min_lat': -10.0,
#             'max_lat': -5.0,
#             'min_lon': 100.0,
#             'max_lon': 120.0
#         },
#         'time_range': {
#             'start': UTCDateTime("2021-06-17T06:11:00"),
#             'end': UTCDateTime("2025-04-11T04:14:00")
#         },
#         'output_dir': "java_earthquake_data",
#         'max_events': 300
#     }
    
#     # Create output directory
#     os.makedirs(config['output_dir'], exist_ok=True)
    
#     # Get earthquake catalog
#     logging.info("Starting catalog retrieval...")
#     catalog = get_java_earthquake_catalog(
#         config['time_range']['start'],
#         config['time_range']['end'],
#         config['region']
#     )
    
#     # Select most recent events
#     catalog.events = sorted(catalog, 
#         key=lambda x: x.origins[0].time, 
#         reverse=True
#     )[:config['max_events']]
    
#     # Save catalog
#     catalog_file = os.path.join(config['output_dir'], "event_catalog.xml")
#     catalog.write(catalog_file, format="QUAKEML")
#     logging.info(f"Catalog saved to {catalog_file}")
    
#     # Download waveform data
#     logging.info(f"Starting waveform download for {len(catalog)} events...")
#     success_count = 0
#     for i, event in enumerate(catalog):
#         logging.info(f"Processing event {i+1}/{len(catalog)}")
#         if download_event_data(event, config['output_dir']):
#             success_count += 1
    
#     logging.info(f"Download completed. {success_count}/{len(catalog)} events downloaded successfully")

# if __name__ == "__main__":
#     main()
